In [1]:
from src.main import generate_corrupted_dataset

data = generate_corrupted_dataset(n_customers=20, years=5)

/Users/paulwilliamson/synthetic_transactions/src/corruptors/base.py:70: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  contract_ammendments = contract_ammendments.stack("customer_id").reset_index(drop=False)
/Users/paulwilliamson/synthetic_transactions/src/corruptors/base.py:87: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["flag_missing_charges"] = df["flag_missing_charges"].fillna(False)


In [2]:
data.columns

Index(['date', 'customer_id', 'order_number', 'item_category_id', 'service_id',
       'variant', 'price', 'quantity', 'final_price', 'new_customer',
       'contract_ammendment', 'condition_1_price', 'condition_2_probability',
       'condition_3_price', 'condition_3_probability',
       'condition_4_probability', 'flag_random_price_change',
       'flag_condition_not_implemented', 'flag_price_change_no_ammendment',
       'flag_missing_charges', 'flag_discrepancy'],
      dtype='object')

In [4]:
data.filter(like="flag_").mean()

flag_random_price_change           0.029695
flag_condition_not_implemented     0.021577
flag_price_change_no_ammendment    0.000371
flag_missing_charges               0.049871
flag_discrepancy                   0.098071
dtype: float64

In [48]:
data_copy = data.copy()

price_changes = data_copy.copy()
price_changes["service_composite"] = price_changes["service_id"].astype(str) + "_" + price_changes["variant"].astype(str)

price_changes = (
    price_changes[["date", "customer_id", "order_number", "service_composite", "price"]]
    .set_index(["date", "customer_id", "order_number", "service_composite"])
)
price_changes = price_changes.groupby(["date", "customer_id", "service_composite"]).mean()

price_changes = price_changes.unstack(["customer_id", "service_composite"]).ffill().bfill().diff()

price_changes = price_changes.stack(["customer_id", "service_composite"])

price_changes.loc[price_changes["price"].abs() > 0, "price"] = True
price_changes.loc[price_changes["price"].abs() == 0, "price"] = False

price_changes = price_changes.groupby(["date", "customer_id"]).max()
price_changes.rename(columns={"price": "CLS_price_change"}, inplace=True)

price_changes.reset_index(drop=False, inplace=True)

output = data_copy.merge(price_changes, on=["date", "customer_id"], how="left")
output["CLS_price_change"] = output["CLS_price_change"].fillna(False)

output = (
    output[["date", "customer_id", "contract_ammendment", "flag_price_change_no_ammendment", "flag_random_price_change", "CLS_price_change"]]
    .groupby(["date", "customer_id"]).max()
)
output.loc[output["contract_ammendment"]==True, "CLS_price_change"] = False

output = output[["flag_price_change_no_ammendment", "flag_random_price_change", "CLS_price_change"]]

/var/folders/41/81ppdf3x65x9r8sjcybxdq380000gn/T/ipykernel_5158/3231558163.py:14: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  price_changes = price_changes.stack(["customer_id", "service_composite"])
/var/folders/41/81ppdf3x65x9r8sjcybxdq380000gn/T/ipykernel_5158/3231558163.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  price_changes.loc[price_changes["price"].abs() > 0, "price"] = True
/var/folders/41/81ppdf3x65x9r8sjcybxdq380000gn/T/ipykernel_5158/3231558163.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Cal

In [51]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred)
    }

output["combined_true"] = output["flag_price_change_no_ammendment"] | output["flag_random_price_change"]

for col in ["flag_price_change_no_ammendment", "flag_random_price_change", "combined_true"]:
    print(f"Metrics for {col}")
    print(metrics(output[col], output["CLS_price_change"]), "\n")

Metrics for flag_price_change_no_ammendment
{'accuracy': 0.6263756294287862, 'recall': 0.7894736842105263, 'precision': 0.0009141324882686331, 'f1': 0.0018261504747991235} 

Metrics for flag_random_price_change
{'accuracy': 0.7482057007450614, 'recall': 0.9924632352941176, 'precision': 0.32902675361082334, 'f1': 0.4942102613391917} 

Metrics for combined_true
{'accuracy': 0.748433548269498, 'recall': 0.9917552216929278, 'precision': 0.3298799439332074, 'f1': 0.49508391640371335} 

